In [1]:
# Force garbage collection
import gc
gc.collect()

41

In [2]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import pyproj
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import roc_curve
import itertools
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import warnings

In [3]:
from sklearn.metrics import confusion_matrix

In [4]:
# check python version and all packages version
def check_python_version():
    import sys
    print("Python version")
    print (sys.version)
    print("Pandas version")
    print(pd.__version__)
    print("Geopandas version")
    print(gpd.__version__)
    print("Xarray version")
    print(xr.__version__)
    print("Pyproj version")
    print(pyproj.__version__)

check_python_version()

Python version
3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
Pandas version
2.2.2
Geopandas version
0.14.2
Xarray version
2023.6.0
Pyproj version
3.6.1


In [6]:
Eval_data = pd.read_parquet('../../Clean_Data/Model_Data/Evaluation/Features_w_Label/2007_features_w_label.parquet')

In [7]:
# convert NWCG_CAUSE_CLASSIFICATION to string
Eval_data['NWCG_CAUSE_CLASSIFICATION'] = Eval_data['NWCG_CAUSE_CLASSIFICATION'].astype(str)
Eval_Human = Eval_data[(Eval_data['NWCG_CAUSE_CLASSIFICATION'].str.contains('Human')) | (Eval_data['IS_FIRE'] == 0)]

In [8]:
# read ../Clean_Data/Model_Data/Downsample/Features_w_Label/features_w_label_downsample_2001_2020.parquet
mod_data = pd.read_parquet('../../Clean_Data/Model_Data/Downsample/Features_w_Label/features_w_label_downsample_2001_2020.parquet')

In [9]:
mod_data['NWCG_CAUSE_CLASSIFICATION'] = mod_data['NWCG_CAUSE_CLASSIFICATION'].astype(str)
mod_Human = mod_data[(mod_data['NWCG_CAUSE_CLASSIFICATION'].str.contains('Human')) | (mod_data['IS_FIRE'] == 0)]

In [10]:
# from day, get Year
mod_Human['Year'] = mod_Human['day'].dt.year
mod_Human = mod_Human[mod_Human['Year'] < 2007]
# print the frequency of each year, order by year
mod_Human['Year'].value_counts().sort_index()

C:\Users\fangshuye\AppData\Local\Temp\ipykernel_25784\3392728417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_Human['Year'] = mod_Human['day'].dt.year


Year
2001    770699
2002    767963
2003    769128
2004    772186
2005    765851
2006    762821
Name: count, dtype: int64

In [11]:
mod_Human.shape, Eval_Human.shape

((4608648, 33), (6259918, 33))

In [12]:
features = ['day','dead_fuel_moisture_1000hr',
                    'dead_fuel_moisture_100hr', 
                    #'pdsi_pre_interpolated',
                    'pdsi', 
                    #'pdsi_class', 
                    'max_air_temperature',
                    'max_relative_humidity', 
                    #'max_wind_speed', 
                    'min_air_temperature',
                    'min_relative_humidity', 
                    'precipitation_amount', 
                    'specific_humidity',
                    'surface_downwelling_shortwave_flux_in_air',
                    'wind_from_direction',
                    'wind_speed', 
                    'wind_direction_category', 
                    'SWE', 
                    'Band1',
                    'LAI',
                    'IS_FIRE',
                    'NWCG_CAUSE_CLASSIFICATION', 
                    'min_FIRE_SIZE', 
                    'max_FIRE_SIZE',
                    'veg_type_details', 
                    'fire_attribute', 
                    'veg', 
                    'slope_avg', 
                    'slope_max',
                    'road_density_km_km2',
                    'Year']

In [13]:
mod_Human = mod_Human[features]
Eval_Human = Eval_Human[features]

In [14]:
cat_columns = ['wind_direction_category','veg']

In [15]:
# for mod_data and Eval_date, use one-hot encoding for each column in cat_columns
mod_Human = pd.get_dummies(mod_Human, columns=cat_columns)
Eval_Human = pd.get_dummies(Eval_Human, columns=cat_columns)

In [16]:
# check the % IS_FIRE
mod_Human['IS_FIRE'].value_counts(normalize=True)

IS_FIRE
0    0.992302
1    0.007698
Name: proportion, dtype: float64

In [17]:
Eval_Human['IS_FIRE'].value_counts(normalize=True)

IS_FIRE
0    0.998576
1    0.001424
Name: proportion, dtype: float64

In [18]:
mod_Human['day'].min(), mod_Human['day'].max()

(Timestamp('2001-01-01 00:00:00'), Timestamp('2006-12-31 00:00:00'))

In [19]:
Eval_Human['day'].min(), Eval_Human['day'].max()

(Timestamp('2007-01-01 00:00:00'), Timestamp('2007-12-31 00:00:00'))

In [20]:
not_in_features = ['day',
                    'wind_from_direction',
                    #'wind_direction_category', 
                    'IS_FIRE',
                    'NWCG_CAUSE_CLASSIFICATION', 
                    'min_FIRE_SIZE', 
                    'max_FIRE_SIZE',
                    'veg_type_details', 
                    'fire_attribute',
                    'Year']

In [21]:
# print final feature sets 
print("features")
print(mod_Human.columns[~mod_Human.columns.isin(not_in_features)])

features
Index(['dead_fuel_moisture_1000hr', 'dead_fuel_moisture_100hr', 'pdsi',
       'max_air_temperature', 'max_relative_humidity', 'min_air_temperature',
       'min_relative_humidity', 'precipitation_amount', 'specific_humidity',
       'surface_downwelling_shortwave_flux_in_air', 'wind_speed', 'SWE',
       'Band1', 'LAI', 'slope_avg', 'slope_max', 'road_density_km_km2',
       'wind_direction_category_N', 'wind_direction_category_NE',
       'wind_direction_category_E', 'wind_direction_category_SE',
       'wind_direction_category_S', 'wind_direction_category_SW',
       'wind_direction_category_W', 'wind_direction_category_NW',
       'veg_Agriculture ', 'veg_Barren ', 'veg_Native Chapparal ',
       'veg_Native Coastal Sage Scrub ', 'veg_Native Conifer Alpine ',
       'veg_Native Conifer Forest ', 'veg_Native Desert ',
       'veg_Native Grassland ', 'veg_Native Inland Scrub ',
       'veg_Native Oak Woodland ', 'veg_Native Wetland ',
       'veg_Non-native forest ', 'veg_No

In [22]:
train_data, test_data = train_test_split(mod_Human, test_size=0.2, shuffle=True, random_state=42)
features = mod_Human.columns[~mod_Human.columns.isin(not_in_features)]
label_col = 'IS_FIRE'

In [23]:
X_train = train_data[features]
y_train = train_data[label_col]

In [28]:
from sklearn.metrics import precision_recall_curve, auc

def calculate_precision_recall(y_true, y_pred_proba, threshold, print_output=False):
    y_pred = (y_pred_proba > threshold).astype(int)
    confusion = confusion_matrix(y_true, y_pred)
    precision = confusion[1, 1] / (confusion[1, 1] + confusion[0, 1])
    recall = confusion[1, 1] / (confusion[1, 1] + confusion[1, 0])
    # F1 score
    f1 = 2 * (precision * recall) / (precision + recall)
    if print_output:
        print(f'Threshold: {threshold:.2f}')
        print(f'Precision: {precision * 100:.2f}%')
        print(f'Recall: {recall * 100:.2f}%')
        print("Confusion Matrix")
        print(pd.DataFrame(confusion, index=['True Neg', 'True Pos'], columns=['Pred Neg', 'Pred Pos']))
    # get TP, TN, FP, FN
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    return TP, TN, FP, FN, precision, recall, f1

def evaluate_model(model, test_data, features, label_col, print_output=False):
    X_test = test_data[features]
    y_test = test_data[label_col]
    # predict the probability of fire
    y_pred = model.predict_proba(X_test)[:, 1]
    # calculate the roc_auc_score
    roc_auc = roc_auc_score(y_test, y_pred)
    # print roc_auc in a sentence
    # print(f"ROC AUC: {roc_auc:.2f}")
    # Calculate precision and recall values
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    # Calculate the area under the precision-recall curve
    auc_pr = auc(recall, precision)
    # print(f"Area Under Precision-Recall Curve (AUC-PR): {auc_pr:.2f}")
    if print_output:
        print(f"ROC AUC: {roc_auc:.2f}")
        print(f"Area Under Precision-Recall Curve (AUC-PR): {auc_pr:.2f}")
    # calculate precision and recall at thresholds 0.5
    TP, TN, FP, FN, precision5, recall5, f15 = calculate_precision_recall(y_test, y_pred, 0.5, print_output)
    return roc_auc, auc_pr, TP, TN, FP, FN, precision5, recall5, f15

In [59]:
results = pd.DataFrame(columns=['model_version','data','roc_auc', 'auc_pr', 'TP', 'TN', 'FP', 'FN', 'precision5', 'recall5', 'f15'])

In [25]:
model1 = xgb.XGBClassifier(eval_metric='logloss', tree_method='hist')
model1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [61]:
# add model1 results to results
temp_result = evaluate_model(model1, test_data, features, label_col)
results = pd.concat([results, 
                     pd.DataFrame([['baseline','validation dat',
                                    temp_result[0],
                                    temp_result[1],
                                    temp_result[2],
                                    temp_result[3],
                                    temp_result[4],
                                    temp_result[5],
                                    temp_result[6],
                                    temp_result[7],
                                    temp_result[8]]], 
                                    columns=results.columns)], ignore_index=True)

C:\Users\fangshuye\AppData\Local\Temp\ipykernel_25784\1760879031.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results,


In [64]:
results

,model_version,data,roc_auc,auc_pr,TP,TN,FP,FN,precision5,recall5,f15
0,baseline,validation dat,0.879667,0.113165,142,914663,49,6876,0.743455,0.020234,0.039395
1,baseline,OOT,0.852427,0.024872,125,6250788,215,8790,0.367647,0.014021,0.027012


In [63]:
# add model1 results to results
temp_result = evaluate_model(model1, Eval_Human, features, label_col)
results = pd.concat([results, 
                     pd.DataFrame([['baseline','OOT',
                                    temp_result[0],
                                    temp_result[1],
                                    temp_result[2],
                                    temp_result[3],
                                    temp_result[4],
                                    temp_result[5],
                                    temp_result[6],
                                    temp_result[7],
                                    temp_result[8]]], 
                                    columns=results.columns)], ignore_index=True)

In [31]:
model2 = xgb.XGBClassifier(eval_metric='aucpr', tree_method='hist')
model2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [32]:
evaluate_model(model2, test_data, features, label_col, print_output=True)

ROC AUC: 0.88
Area Under Precision-Recall Curve (AUC-PR): 0.11
Threshold: 0.50
Precision: 74.35%
Recall: 2.02%
Confusion Matrix
          Pred Neg  Pred Pos
True Neg    914663        49
True Pos      6876       142


(0.8796674107635724,
 0.11316521717135991,
 142,
 914663,
 49,
 6876,
 0.743455497382199,
 0.020233684810487318,
 0.03939520044388958)

In [33]:
evaluate_model(model2, Eval_Human, features, label_col, print_output=True)

ROC AUC: 0.85
Area Under Precision-Recall Curve (AUC-PR): 0.02
Threshold: 0.50
Precision: 36.76%
Recall: 1.40%
Confusion Matrix
          Pred Neg  Pred Pos
True Neg   6250788       215
True Pos      8790       125


(0.8524268859702434,
 0.024872218262008583,
 125,
 6250788,
 215,
 8790,
 0.36764705882352944,
 0.014021312394840156,
 0.02701242571582928)

In [74]:
weights = train_data['min_FIRE_SIZE']
percentile_ranks = weights.rank(pct=True)

In [77]:
model3 = xgb.XGBClassifier(eval_metric='logloss', tree_method='hist')
model3.fit(X_train, y_train, sample_weight=percentile_ranks)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [78]:
# add model1 results to results
temp_result = evaluate_model(model3, test_data, features, label_col)
results = pd.concat([results, 
                     pd.DataFrame([['weight using rank','validation dat',
                                    temp_result[0],
                                    temp_result[1],
                                    temp_result[2],
                                    temp_result[3],
                                    temp_result[4],
                                    temp_result[5],
                                    temp_result[6],
                                    temp_result[7],
                                    temp_result[8]]], 
                                    columns=results.columns)], ignore_index=True)

In [79]:
# add model1 results to results
temp_result = evaluate_model(model3, Eval_Human, features, label_col)
results = pd.concat([results, 
                     pd.DataFrame([['weight using rank','OOT',
                                    temp_result[0],
                                    temp_result[1],
                                    temp_result[2],
                                    temp_result[3],
                                    temp_result[4],
                                    temp_result[5],
                                    temp_result[6],
                                    temp_result[7],
                                    temp_result[8]]], 
                                    columns=results.columns)], ignore_index=True)

In [80]:
# print result interactively, so that I can order by columns 
results

,model_version,data,roc_auc,auc_pr,TP,TN,FP,FN,precision5,recall5,f15
0,baseline,validation dat,0.879667,0.113165,142,914663,49,6876,0.743455,0.020234,0.039395
1,baseline,OOT,0.852427,0.024872,125,6250788,215,8790,0.367647,0.014021,0.027012
2,add weight,validation dat,0.879421,0.110026,150,914668,44,6868,0.773196,0.021374,0.041597
3,add weight,OOT,0.855091,0.024034,120,6250793,210,8795,0.363636,0.013460,0.025960
4,weight using rank,validation dat,0.878257,0.111294,153,914656,56,6865,0.732057,0.021801,0.042341
5,weight using rank,OOT,0.851106,0.023377,122,6250763,240,8793,0.337017,0.013685,0.026302
